In [82]:
import pandas as pd
import json
import ast

year = 2022

In [83]:
data = pd.read_csv(f'..\data\clean\sustainability-report-{year}-squad-format.csv', sep=";")

In [84]:
df1 = pd.DataFrame(data['answers'].apply(ast.literal_eval).values.tolist())
df1.columns = 'answers.'+ df1.columns
col = data.columns.difference(['answers'])
data = pd.concat([data[col], df1],axis=1)

In [85]:
json_data = {"version":1, "data":[]}

In [86]:
for row in data.iterrows():
    row_data = row[1]
    id = row_data["id"]
    question = row_data["question"]
    context = row_data["context"]
    answer = row_data["answers.text"][0]
    answer_start = row_data["answers.answer_start"][0]
    single_row = {
        "title": f"Title {id}",
        "paragraphs": [
            {
                "context": f"{context}",
                "qas": [
                    {
                        "id": f"{id}",
                        "question": f"{question}",
                        "answers": [
                            {
                                "text": f"{answer}",
                                "answer_start": answer_start
                            }
                        ],
                        "is_impossible": True
                    }
                ]
            }
        ]
    }
    json_data["data"].append(single_row)

In [87]:
# Serializing json
json_object = json.dumps(json_data, indent=4)
 
# Writing to sample.json
with open(f'..\data\clean\sustainability-report-{year}-squad-format.json', "w") as outfile:
    outfile.write(json_object)